In [ ]:
# Kuvaava otsikko :)

JAMK examples #5: 05_ts_sales_data_categories

Ladataan käytettävät paketit.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Luetaan käytettävä aineisto:

In [3]:
csvUrl = "https://raw.githubusercontent.com/RaimoHaikari/paikkatieto/master/misc/sales_data.csv"
df = pd.read_csv(csvUrl)

df.dtypes

date                object
category            object
sales                int64
price              float64
customer_age         int64
customer_gender     object
dtype: object

Muunnetaan date-sarake aikaleimaksi.

In [4]:
# Convert the 'date' column to the datetime
df['date'] = pd.to_datetime(df['date'])

Lasketaan päivittäiset myynnit omaan sarakkeeseensa. Lopputulemana halutaan kehikko, jossa:

- myynnit ja päivät näkyvät omina muuttujinaan 
- indeksinä käytetään juoksevaa numerointia

In [8]:
daily_sales =  df.resample('D', on='date')['sales'].sum().reset_index()

tähän jääätiiiiiin

In [ ]:
# Group the data by category and day, calculate total sales, and reset index
# daily_sales_by_category = df.groupby(['category', pd.Grouper(key='date', freq='D')])['sales'].sum(
#    numeric_only=True).reset_index()